In [ ]:
import pandas as pd
import os
from datetime import datetime
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
def filesearch(name=''):
    currentd = os.getcwd()
    for root, dirs, files in os.walk(currentd):
        if name in files:
            return os.path.join(root,name)
        if name in dirs:
            return os.path.join(root,name)

def dateform(d):
    date = str(d)
    date = date.split(' ')[0]
    date = date.split('-')
    date = date[0] + date[1]
    return(date)
        
def valform(v):
    temp = v.replace('.', '').replace(',','.')
    try:
        if '-' in temp:
            temp =-float(temp)
        else:
            temp = float(temp)
    except ValueError:
        print('UnconvertableAmount:', v)
        pass
    return(temp)
        
def filelist(mode = 1):
    check_list = []
    for root, dirs, files in os.walk(filesearch('2021')):
        check_list.extend(files)
    CL = []
    for check in check_list:
        if '2021' in check:
            name = check.split(' ')
            temp1 = name[0]
            temp2 = ' '.join(x for x in name[1:-1])
            temp3 = valform(name[-1].split('.')[0])
            if (len(name)<3 or type(temp3) == str):
                continue
            CL.append((temp1,temp2,temp3))
    if mode == 0:
        return(check_list)
    return(CL)

def writelist(check_list):
    checks=check_list
    checks=map(lambda x:x+'\n', checks)
    with open('UsedntChecks.txt','w') as txt:
        txt.writelines(checks)

In [ ]:
'''Index(['Naziv nal./prej.', 'Račun', 'Znesek', 'Opis', 'Razlog zavrnitve',
       'Ref. prej.', 'Ref. nalog.', 'Datum knj.', 'Datum val.'],
      dtype='object')'''

name_list = pd.read_csv(filesearch('name_list.txt'), sep=' = ', engine='python')
file = pd.read_excel(filesearch('2021 izpis.xlsx'))
file.insert(file.columns.size, 'file', None)
check_list = filelist()
#file

In [ ]:
def znesekfix(file):
    fixed = file
    for i, value in fixed.iterrows():
        if type(value['Znesek']) == str:
            value['Znesek'] = valform(value['Znesek'])
            fixed.iloc[i] = value
    return(fixed)

def alias(file, name_list):
    df = file
    aliases = {}
    for i, row in name_list.iterrows():
        aliases[row['old']] = row['new']
    for i, row in df.iterrows():
        if row['Naziv nal./prej.'] in aliases:
            row['Naziv nal./prej.'] = aliases[row['Naziv nal./prej.']]
        df.iloc[i] = row
    return(df)

def merge(file, check_list, mode = 1):
    merged = file
    usedntch = check_list
    for i, row in file.iterrows():
        for check in check_list:
            cond1 =-check[-1] == row['Znesek']
            cond2 = check[1]  == row['Naziv nal./prej.']
            cond3 = check[0][:6] == dateform(row['Datum val.'])
            if cond1 and cond2 and cond3:
                name = ' '.join(x for x in check[:-1]) + ' ' +str(check[-1])
                row['file'] = name
                merged.iloc[i] = row
                usedntch.remove(check)
    if mode == 0:
        return((merged, usedntch))
    return(merged)

In [ ]:
date = datetime.now().strftime('%Y%m%d')
DF = alias(file, name_list)
DF = znesekfix(DF)
DF = DF.loc[DF['Naziv nal./prej.'] != 'IGNORE'].reset_index(drop=True)
DF = merge(DF, check_list, mode=0)
#DF[0].to_excel(str(date) +'CompilerResult.xlsx')
DF[1]